<a href="https://colab.research.google.com/github/georgejordan3/IBM_Capstone/blob/main/Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Segmenting and Clustering Neighboorhoods in Toronto

George Jordan <br>
IBM Applied Data Science Capstone <br>
Last Updated: 2-21-21 <br>

This notebook will demonstrate the methods I used to scrape a wikipedia article for data to be implemented into a localized pandas dataframe.

## Scraping and Creating the Dataframe

First, I will import the libraries necessary for scraping the URL.

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

Using the requests library, I store the html data from the URL locally in the variable 'html_data'.

In [3]:
html_data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

I then used BeautifulSoup to parse the html and select the table that contained the postal code data.

In [ ]:
soup = BeautifulSoup(html_data.content, 'html.parser')
postal_table = soup.find("table", {'class':"wikitable"})
postal_table

In [45]:
amazon_data = pd.DataFrame(columns=["Date", "Open", "High"])
for row in soup.find("tbody").find_all("tr"):
    col = row.find_all("span")
    date = (col[0].text)
    Open = (col[1].text)
    high = (col[2].text)
    low = (col[3].text)
    close = (col[4].text)
    adj_close = (col[5].text)
    volume = (col[6].text)
    amazon_data = amazon_data.append({"Date":date, "Open":Open, "High":high}, ignore_index=True)

ValueError: ignored